In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/summerization-dataset/test (1).csv


In [2]:
#import dataset into a data frame
df=pd.read_csv("/kaggle/input/summerization-dataset/test (1).csv")

In [3]:
df.head()

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


In [4]:
ROUGE1=[]
ROUGE2=[]
ROUGEL=[]

In [5]:
pip install rouge-score


  Preparing metadata (setup.py) ... - done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a04849ce8099f534634860806ef9c94cd7f289a871a9aa565e90f92d64bad4dc
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.


**T5 MODEL**

In [6]:
import matplotlib.pyplot as plt
from transformers import T5ForConditionalGeneration, T5Tokenizer
from rouge_score import rouge_scorer

def summarize_text_t5(text, max_length=150):
    model_name = "t5-small"  # You can use other models like "t5-base" or "t5-large"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    input_text = "summarize: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    summary_ids = model.generate(inputs, max_length=max_length, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def evaluate_summary(generated_summary, reference_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_summary, generated_summary)
    return scores


x=0
y=0
z=0
for i in range(0,20):
# Generate summary
    generated_summary = summarize_text_t5(df.article[i])
    print("GENERATED SUMMARY:")
    print("------------------")
    print(generated_summary)
    print("REFERENCE SUMMARY:")
    print("------------------")
    reference_summary = df.highlights[i]
    print(reference_summary)
    scores = evaluate_summary(generated_summary, reference_summary)
    x=x+(scores['rouge1'].precision+scores['rouge1'].recall+scores['rouge1'].fmeasure)/3
    y=y+(scores['rouge2'].precision+scores['rouge2'].recall+scores['rouge2'].fmeasure)/3
    z=z+(scores['rougeL'].precision+scores['rougeL'].recall+scores['rougeL'].fmeasure)/3

#Average score

ROUGE1.append(x/20)
ROUGE2.append(y/20)
ROUGEL.append(z/20)

# Print the scores
print("ROUGE-1 Precision: {:.2f}".format(scores['rouge1'].precision))
print("ROUGE-1 Recall: {:.2f}".format(scores['rouge1'].recall))
print("ROUGE-1 F1 Score: {:.2f}".format(scores['rouge1'].fmeasure))
print("ROUGE-2 Precision: {:.2f}".format(scores['rouge2'].precision))
print("ROUGE-2 Recall: {:.2f}".format(scores['rouge2'].recall))
print("ROUGE-2 F1 Score: {:.2f}".format(scores['rouge2'].fmeasure))
print("ROUGE-L Precision: {:.2f}".format(scores['rougeL'].precision))
print("ROUGE-L Recall: {:.2f}".format(scores['rougeL'].recall))
print("ROUGE-L F1 Score: {:.2f}".format(scores['rougeL'].fmeasure))



tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

GENERATED SUMMARY:
------------------
Prasanna ‘Nick’ Arulchelvam ran and jumped through an open door of their van. but he was pushed out and hit the ground as the vehicle sped away. he was jailed for 11 years after admitting all he wanted was 'a few cheap packets of cigarettes'
REFERENCE SUMMARY:
------------------
Prasanna Arulchelvam leapt into van as it sped away but was pushed out .
His head hit the ground with a 'nasty crunch' and he died 11 days later .
A gang tried to steal cigarettes from the victim's van before he gave chase .
All three have now been jailed, including the man who pushed Mr Prasanna .


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GENERATED SUMMARY:
------------------
a fiery sunset greeted people in Washington. the deep reddish color caught the eye of a native. the fires started in southeastern Siberia, by farmers burning grass.
REFERENCE SUMMARY:
------------------
Smoke from massive fires in Siberia created fiery sunsets in the Pacific Northwest .
Atmospheric winds carried smoke from the wildfires across the Pacific Ocean .
Smoke particles altered wavelengths from the sun, creating a more intense color .


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GENERATED SUMMARY:
------------------
camel - the first intact camel skeleton found in central Europe - may have been left in the town of Tulln for trading after the siege of Vienna in 1683. genetic analysis of the animal show that it was a Bactrian-dromedary hybrid – a breed popular in the Ottoman army at the time. it was male, around seven years old and most likely castrated.
REFERENCE SUMMARY:
------------------
Scientists say camel was left in Tulln after the 1683 siege of Vienna .
It would have shocked residents as camels were an alien species .
'They didn't know what to feed it or whether one could eat it,' study said .
Ottoman army used camels for transportation and as riding animals .
ROUGE-1 Precision: 0.32
ROUGE-1 Recall: 0.40
ROUGE-1 F1 Score: 0.36
ROUGE-2 Precision: 0.08
ROUGE-2 Recall: 0.10
ROUGE-2 F1 Score: 0.09
ROUGE-L Precision: 0.21
ROUGE-L Recall: 0.26
ROUGE-L F1 Score: 0.23


In [7]:
print(df.article[1])

A drunk teenage boy had to be rescued by security after jumping into a lions' enclosure at a zoo in western India. Rahul Kumar, 17, clambered over the enclosure fence at the Kamla Nehru Zoological Park in Ahmedabad, and began running towards the animals, shouting he would 'kill them'. Mr Kumar explained afterwards that he was drunk and 'thought I'd stand a good chance' against the predators. Next level drunk: Intoxicated Rahul Kumar, 17, climbed into the lions' enclosure at a zoo in Ahmedabad and began running towards the animals shouting 'Today I kill a lion!' Mr Kumar had been sitting near the enclosure when he suddenly made a dash for the lions, surprising zoo security. The intoxicated teenager ran towards the lions, shouting: 'Today I kill a lion or a lion kills me!' A zoo spokesman said: 'Guards had earlier spotted him close to the enclosure but had no idea he was planing to enter it. 'Fortunately, there are eight moats to cross before getting to where the lions usually are and he

In [8]:
print(ROUGE1,ROUGE2,ROUGEL)

[0.41511746791037096] [0.18931591883449986] [0.2910678497836114]


**2.PEGASUS**

In [9]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

def summarize_text_pegasus(text, model_name="google/pegasus-xsum", max_length=150):
    tokenizer = PegasusTokenizer.from_pretrained(model_name)
    model = PegasusForConditionalGeneration.from_pretrained(model_name)

    inputs = tokenizer(text, truncation=True, padding="longest", return_tensors="pt")
    summary_ids = model.generate(inputs["input_ids"], max_length=max_length, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def evaluate_summary(generated_summary, reference_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_summary, generated_summary)
    return scores

x=0
y=0
z=0
for i in range(0,20):
# Generate summary
    generated_summary = summarize_text_pegasus(df.article[i])
    print("GENERATED SUMMARY:")
    print("------------------")
    print(generated_summary)
    print("REFERENCE SUMMARY:")
    print("------------------")
    reference_summary = df.highlights[i]
    print(reference_summary)
    scores = evaluate_summary(generated_summary, reference_summary)
    x=x+(scores['rouge1'].precision+scores['rouge1'].recall+scores['rouge1'].fmeasure)/3
    y=y+(scores['rouge2'].precision+scores['rouge2'].recall+scores['rouge2'].fmeasure)/3
    z=z+(scores['rougeL'].precision+scores['rougeL'].recall+scores['rougeL'].fmeasure)/3

#Average score

ROUGE1.append(x/20)
ROUGE2.append(y/20)
ROUGEL.append(z/20)


tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

...

GENERATED SUMMARY:
------------------
A man has been jailed for 11 years for the manslaughter of a shopkeeper who fell from his van while chasing a gang of thieves in a cash and carry car park.
REFERENCE SUMMARY:
------------------
Prasanna Arulchelvam leapt into van as it sped away but was pushed out .
His head hit the ground with a 'nasty crunch' and he died 11 days later .
A gang tried to steal cigarettes from the victim's van before he gave chase .
All three have now been jailed, including the man who pushed Mr Prasanna .


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GENERATED SUMMARY:
------------------
All images are copyrighted. The reason people are seeing an intense red sunset is a result of smoke particles filtering out the shorter wavelength colors from the sunlight like greens, blues, yellows and purples, KOMO-TV said.
REFERENCE SUMMARY:
------------------
Smoke from massive fires in Siberia created fiery sunsets in the Pacific Northwest .
Atmospheric winds carried smoke from the wildfires across the Pacific Ocean .
Smoke particles altered wavelengths from the sun, creating a more intense color .


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GENERATED SUMMARY:
------------------
The complete skeleton of an 'alien' 17th-century camel that was likely used in the second Ottoman-Habsburg war has been found in Austria.
REFERENCE SUMMARY:
------------------
Scientists say camel was left in Tulln after the 1683 siege of Vienna .
It would have shocked residents as camels were an alien species .
'They didn't know what to feed it or whether one could eat it,' study said .
Ottoman army used camels for transportation and as riding animals .


**3.BERT MODEL**

In [10]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

def evaluate_summary(generated_summary, reference_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_summary, generated_summary)
    return scores
def summerize_bertmodel(text):
# Preprocessing
    sentences = text.split(". ")
    inputs = [f"[CLS] {sentence} [SEP]" for sentence in sentences]

# Load pre-trained model and tokenizer
    model_name = "bert-base-uncased"
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
    # Tokenize and predict
    inputs = tokenizer(inputs, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**inputs)
    scores = outputs.logits[:, 1]
    # Select top sentences based on scores
    num_sentences = 2
    top_sentence_indices = torch.topk(scores, num_sentences).indices
    summary = " ".join([sentences[i] for i in top_sentence_indices])
    return summary
x=0
y=0
z=0
for i in range(0,20):
# Generate summary
    generated_summary = summerize_bertmodel(df.article[i])
    print("GENERATED SUMMARY:")
    print("------------------")
    print(generated_summary)
    print("REFERENCE SUMMARY:")
    print("------------------")
    reference_summary = df.highlights[i]
    print(reference_summary)
    scores = evaluate_summary(generated_summary, reference_summary)
    x=x+(scores['rouge1'].precision+scores['rouge1'].recall+scores['rouge1'].fmeasure)/3
    y=y+(scores['rouge2'].precision+scores['rouge2'].recall+scores['rouge2'].fmeasure)/3
    z=z+(scores['rougeL'].precision+scores['rougeL'].recall+scores['rougeL'].fmeasure)/3

#Average score

ROUGE1.append(x/20)
ROUGE2.append(y/20)
ROUGEL.append(z/20)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

GENERATED SUMMARY:
------------------
Four years ago he married Shayanthy, 30, and the couple lived in Grandfield Avenue, Watford with their young son. The day before Mr Prasanna was killed, the two O’Driscolls had carried out thefts and attempted thefts at Booker cash and carry stores in Dagenham, Chelmsford, Colchester and Ipswich
REFERENCE SUMMARY:
------------------
Prasanna Arulchelvam leapt into van as it sped away but was pushed out .
His head hit the ground with a 'nasty crunch' and he died 11 days later .
A gang tried to steal cigarettes from the victim's van before he gave chase .
All three have now been jailed, including the man who pushed Mr Prasanna .


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GENERATED SUMMARY:
------------------
As the smoke starts to dissipate, air quality will get better and these fiery sunsets will lose their reddish hue. (CNN)A fiery sunset greeted people in Washington Sunday
REFERENCE SUMMARY:
------------------
Smoke from massive fires in Siberia created fiery sunsets in the Pacific Northwest .
Atmospheric winds carried smoke from the wildfires across the Pacific Ocean .
Smoke particles altered wavelengths from the sun, creating a more intense color .


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GENERATED SUMMARY:
------------------
'The animal was certainly exotic for the people of Tulln The remarkable find was made during an archaeological dig that took place amid preparations for a new shopping centre in the town
REFERENCE SUMMARY:
------------------
Scientists say camel was left in Tulln after the 1683 siege of Vienna .
It would have shocked residents as camels were an alien species .
'They didn't know what to feed it or whether one could eat it,' study said .
Ottoman army used camels for transportation and as riding animals .


**4.text rank**

In [11]:
import pandas as pd
import nltk
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

def evaluate_summary(generated_summary, reference_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_summary, generated_summary)
    return scores
def preprocess_text(text):
    sentences = sent_tokenize(text)
    return sentences

def build_similarity_matrix(sentences, stop_words):
    vectorizer = TfidfVectorizer(stop_words=stop_words)
    tfidf_matrix = vectorizer.fit_transform(sentences)
    similarity_matrix = cosine_similarity(tfidf_matrix)
    return similarity_matrix

def text_rank(sentences, similarity_matrix):
    nx_graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    return ranked_sentences

def summarize_text(text, num_sentences=2):
    stop_words = list(stopwords.words('english'))  # Convert stop words to list
    sentences = preprocess_text(text)
    similarity_matrix = build_similarity_matrix(sentences, stop_words)
    ranked_sentences = text_rank(sentences, similarity_matrix)
    summary = ' '.join([sent[1] for sent in ranked_sentences[:num_sentences]])
    return summary

x=0
y=0
z=0

for i in range(0,20):
# Generate summary
    generated_summary = summarize_text(df.article[i],num_sentences=2)
    print("GENERATED SUMMARY:")
    print("------------------")
    print(generated_summary)
    print("REFERENCE SUMMARY:")
    print("------------------")
    reference_summary = df.highlights[i]
    print(reference_summary)
    scores = evaluate_summary(generated_summary, reference_summary)
    x=x+(scores['rouge1'].precision+scores['rouge1'].recall+scores['rouge1'].fmeasure)/3
    y=y+(scores['rouge2'].precision+scores['rouge2'].recall+scores['rouge2'].fmeasure)/3
    z=z+(scores['rougeL'].precision+scores['rougeL'].recall+scores['rougeL'].fmeasure)/3

#Average score

ROUGE1.append(x/20)
ROUGE2.append(y/20)
ROUGEL.append(z/20)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
GENERATED SUMMARY:
------------------
But these tests are conducted using planes with 31 inches between each row of seats, a standard which on some airlines has decreased, reported the Detroit News. British Airways has a seat pitch of 31 inches, while easyJet has 29 inches, Thomson's short haul seat pitch is 28 inches, and Virgin Atlantic's is 30-31.
REFERENCE SUMMARY:
------------------
Experts question if  packed out planes are putting passengers at risk .
U.S consumer advisory group says minimum space must be stipulated .
Safety tests conducted on planes with more leg room than airlines offer .
GENERATED SUMMARY:
------------------
Next level drunk: Intoxicated Rahul Kumar, 17, climbed into the lions' enclosure at a zoo in Ahmedabad and began runn

**5.TF-IDF**

In [12]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')

def evaluate_summary(generated_summary, reference_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_summary, generated_summary)
    return scores

def preprocess_text(text):
    sentences = sent_tokenize(text)
    return sentences

def compute_tfidf(sentences):
    stop_words = stopwords.words('english')
    vectorizer = TfidfVectorizer(stop_words=stop_words)
    tfidf_matrix = vectorizer.fit_transform(sentences)
    return tfidf_matrix

def rank_sentences(sentences, tfidf_matrix):
    sentence_scores = np.sum(tfidf_matrix.toarray(), axis=1)
    ranked_sentence_indices = np.argsort(sentence_scores)[::-1]
    ranked_sentences = [sentences[i] for i in ranked_sentence_indices]
    return ranked_sentences

def summarize_text(text, num_sentences=2):
    sentences = preprocess_text(text)
    tfidf_matrix = compute_tfidf(sentences)
    ranked_sentences = rank_sentences(sentences, tfidf_matrix)
    summary = ' '.join(ranked_sentences[:num_sentences])
    return summary

x=0
y=0
z=0



for i in range(0,20):
# Generate summary
    generated_summary = summarize_text(df.article[i],num_sentences=2)
    print("GENERATED SUMMARY:")
    print("------------------")
    print(generated_summary)
    print("REFERENCE SUMMARY:")
    print("------------------")
    reference_summary = df.highlights[i]
    print(reference_summary)
    scores = evaluate_summary(generated_summary, reference_summary)
    x=x+(scores['rouge1'].precision+scores['rouge1'].recall+scores['rouge1'].fmeasure)/3
    y=y+(scores['rouge2'].precision+scores['rouge2'].recall+scores['rouge2'].fmeasure)/3
    z=z+(scores['rougeL'].precision+scores['rougeL'].recall+scores['rougeL'].fmeasure)/3

#Average score

ROUGE1.append(x/20)
ROUGE2.append(y/20)
ROUGEL.append(z/20)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
GENERATED SUMMARY:
------------------
This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. But could crowding on planes lead to more serious issues than fighting for space in the overhead lockers, crashing elbows and seat back kicking?
REFERENCE SUMMARY:
------------------
Experts question if  packed out planes are putting passengers at risk .
U.S consumer advisory group says minimum space must be stipulated .
Safety tests conducted on planes with more leg room than airlines offer .
GENERATED SUMMARY:
------------------
Brave fool: Fortunately, Mr Kumar  fe

**6.LSA**

In [13]:
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.tokenize import sent_tokenize

#nltk.download('punkt')

def summarize_text_lsa(text, num_sentences=3):
    # Step 1: Split the text into sentences
    sentences = sent_tokenize(text)
    if len(sentences) <= num_sentences:
        return text

    # Step 2: Construct a TF-IDF matrix for the sentences
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)

    # Step 3: Apply Singular Value Decomposition (SVD)
    svd = TruncatedSVD(n_components=num_sentences)
    svd_matrix = svd.fit_transform(tfidf_matrix.T)

    # Step 4: Rank the sentences based on their importance
    sentence_scores = svd_matrix.sum(axis=0)
    ranked_sentence_indices = np.argsort(sentence_scores)[::-1][:num_sentences]

    # Step 5: Select the top-ranked sentences to form the summary
    summary = [sentences[i] for i in sorted(ranked_sentence_indices)]
    return ' '.join(summary)

def evaluate_summary(generated_summary, reference_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_summary, generated_summary)
    return scores

x=0
y=0
z=0

for i in range(0,20):
# Generate summary
    generated_summary = summarize_text_lsa(df.article[i],num_sentences=2)
    print("GENERATED SUMMARY:")
    print("------------------")
    print(generated_summary)
    print("REFERENCE SUMMARY:")
    print("------------------")
    reference_summary = df.highlights[i]
    print(reference_summary)
    scores = evaluate_summary(generated_summary, reference_summary)
    x=x+(scores['rouge1'].precision+scores['rouge1'].recall+scores['rouge1'].fmeasure)/3
    y=y+(scores['rouge2'].precision+scores['rouge2'].recall+scores['rouge2'].fmeasure)/3
    z=z+(scores['rougeL'].precision+scores['rougeL'].recall+scores['rougeL'].fmeasure)/3

#Average score

ROUGE1.append(x/20)
ROUGE2.append(y/20)
ROUGEL.append(z/20)




GENERATED SUMMARY:
------------------
Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk.
REFERENCE SUMMARY:
------------------
Experts question if  packed out planes are putting passengers at risk .
U.S consumer advisory group says minimum space must be stipulated .
Safety tests conducted on planes with more leg room than airlines offer .
GENERATED SUMMARY:
------------------
A drunk teenage boy had to be rescued by security after jumping into a lions' enclosure at a zoo in western India. Rahul Kumar, 17, clambered over the enclosure fence at the Kamla Nehru Zoological Park in Ahmedabad, and began running towards the animals, shouting he would 'kill them'.
REFERENCE SUMMARY:
------------------
Drunk teenage boy climbed into lion enclosure at zoo in west India .
Rahul Kumar, 17, ran towards animals shouting 'Today I kill 

**EVALUATION TABLE**

In [14]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [15]:
averge=[ROUGE1[i]+ROUGE2[i]+ROUGEL[i] for i in range(6)]

In [16]:
data = {
    "ROUGE Type": ['T5','PEAGUS','BERT','TF_IDF','TEXTRANK','LSA'],
    "ROUGE-1": ROUGE1,
    "ROUGE-2": ROUGE2,
    "ROUGE-L": ROUGEL,
    "AVGSCORE":averge
}



In [17]:
import pandas as pd

d=pd.DataFrame(data)
print(d)

  ROUGE Type   ROUGE-1   ROUGE-2   ROUGE-L  AVGSCORE
0         T5  0.415117  0.189316  0.291068  0.895501
1     PEAGUS  0.275255  0.076682  0.177588  0.529526
2       BERT  0.264803  0.077270  0.166546  0.508619
3     TF_IDF  0.345053  0.120582  0.222272  0.687907
4   TEXTRANK  0.343228  0.115169  0.195959  0.654357
5        LSA  0.414949  0.191535  0.284402  0.890886


In [18]:
ROUGE1

[0.41511746791037096,
 0.27525509389096403,
 0.2648034986405693,
 0.3450528830823487,
 0.34322804823469955,
 0.41494890205964385]